# Tutorial 5: Solving Linear Systems

In previous tutorials, we've built `HilbertSpace`s and the `LinearOperator`s that act on them. Now, we'll focus on the final piece of the puzzle: solving linear systems. The `linear_solvers` module provides a unified interface for inverting `LinearOperator`s.

### The Solver Design

The solvers in this library are designed as callable classes. You first create an instance of a solver (e.g., `CholeskySolver`), and then you call that instance with the operator you want to invert. The result is a *new* `LinearOperator` that represents the inverse of the original.

`A_inv = solver(A)`

This "functional" approach is very flexible and fits well with the operator algebra.

In this tutorial, we will:
1.  Set up a simple linear system in `EuclideanSpace`.
2.  Solve it using a **direct solver** (`CholeskySolver`).
3.  Solve it again using an **iterative solver** (`CGSolver`).
4.  Discuss when to choose each type of solver.
5.  Demonstrate how **preconditioning** can accelerate iterative solvers.

In [ ]:
# To run in colab, uncomment the line below to install pygeoinf. 
#%pip install pygeoinf --quiet

import numpy as np
import time
import pygeoinf as inf


# For reproducibility
np.random.seed(0)

## 1. Setting up a Linear System

Let's create a simple, well-posed linear system to solve. We'll define a positive-definite symmetric operator `A` and a right-hand-side vector `y`. Our goal is to find the vector `x` such that `A(x) = y`.

In [ ]:
# Create a 100-dimensional space
space = inf.EuclideanSpace(100)

# Create a symmetric, positive-definite matrix for our operator
M = np.random.randn(100, 100)
M_symm = M @ M.T + 0.1 * np.identity(100) # Ensure it's positive-definite

# Create the LinearOperator
A = inf.LinearOperator.from_matrix(space, space, M_symm, galerkin=True)

# Create a "true" solution x_true and compute the right-hand-side y
x_true = space.random()
y = A(x_true)

print(f"Setup complete. We will solve for x in A(x) = y.")

## 2. Direct Solvers

Direct solvers, like `LUSolver` or `CholeskySolver`, work by first factorizing the matrix representation of the operator (e.g., into lower and upper triangular matrices) and then solving the system via substitution.

* **Pros:** They are exact (to machine precision) and very robust.
* **Cons:** They require forming and storing the entire matrix, which can be prohibitively slow and memory-intensive for large problems.

The `CholeskySolver` is an excellent choice for symmetric, positive-definite operators.

In [ ]:
# 1. Create an instance of the solver
cholesky_solver = inf.CholeskySolver(galerkin=True)

# 2. Call the solver on the operator to get its inverse
A_inv_direct = cholesky_solver(A)

# 3. Apply the inverse operator to y to find the solution x
start_time = time.time()
x_direct = A_inv_direct(y)
end_time = time.time()

print(f"Direct solve took: {end_time - start_time:.6f} seconds.")
error_direct = space.norm(space.subtract(x_direct, x_true))
print(f"Error in direct solution: {error_direct:.2e}")

## 3. Iterative Solvers

Iterative solvers, like `CGSolver` (Conjugate Gradient), start with an initial guess and progressively refine the solution in a series of steps until a desired tolerance is met.

* **Pros:** They do not require the matrix to be stored explicitly; they only need the *action* of the operator on a vector. This makes them extremely efficient in terms of memory for very large problems.
* **Cons:** They are approximate and their performance can depend heavily on the properties of the operator (its "condition number").

The `CGSolver` is a matrix-free implementation of Conjugate Gradient, suitable for self-adjoint operators.

In [ ]:
# 1. Create an instance of the solver with a desired tolerance
cg_solver = inf.CGSolver(rtol=1e-9)

# 2. Get the inverse operator
A_inv_iterative = cg_solver(A)

# 3. Apply the inverse to solve for x
start_time = time.time()
x_iterative = A_inv_iterative(y)
end_time = time.time()

print(f"Iterative solve took: {end_time - start_time:.6f} seconds.")
error_iterative = space.norm(space.subtract(x_iterative, x_true))
print(f"Error in iterative solution: {error_iterative:.2e}")

## 4. Choosing the Right Solver

The choice of solver is a practical trade-off:

* For **small to medium-sized problems** (e.g., dimensions up to a few thousand), a **direct solver** is often the best choice. It's robust, easy to use, and often faster because the overhead of factorizing the matrix is less than the cost of many iterations.
* For **large-scale problems** (e.g., function spaces with millions of degrees of freedom), an **iterative solver** is the *only* feasible option. Forming a million-by-million dense matrix is impossible on modern computers.


## 5. Preconditioning

The performance of iterative solvers can be dramatically improved with a **preconditioner**. A preconditioner is an operator `P` that is an approximation of the inverse of `A`. The solver then solves a modified, better-conditioned system.

A perfect preconditioner would be `P = A_inv`, but that's what we're trying to find! A good preconditioner is a cheap approximation of the inverse. The simplest choice is the **Jacobi preconditioner**, which is just the inverse of the diagonal of `A`.

In [ ]:
# 1. Create the Jacobi preconditioner
# Get the diagonal of the matrix of A
diag_A = A.matrix(dense=True, galerkin=True).diagonal()
# The preconditioner is the inverse of the diagonal
preconditioner = inf.DiagonalSparseMatrixLinearOperator.from_diagonal_values(space, space, 1.0 / diag_A, galerkin=True)

# 2. Create a new inverse operator, this time providing the preconditioner
A_inv_preconditioned = cg_solver(A, preconditioner=preconditioner)

# 3. Solve the system again
start_time = time.time()
x_preconditioned = A_inv_preconditioned(y)
end_time = time.time()

print(f"Preconditioned iterative solve took: {end_time - start_time:.6f} seconds.")
error_preconditioned = space.norm(space.subtract(x_preconditioned, x_true))
print(f"Error in preconditioned solution: {error_preconditioned:.2e}")

print("\nNote: For this simple, well-conditioned problem, the speed-up is minimal,")
print("but for ill-conditioned problems, a good preconditioner can reduce the")
print("solve time by orders of magnitude.")